# Perform GSEA using GSEAPY  

Following the potocol defined here: https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html#use-gsea-command-or-gsea


In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac

import gseapy as gp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
gp.__version__

'0.9.9'

In [ ]:
patchseq_genes = pd.read_csv('../dat/1908/20190802_patchseq_genesets.csv',skiprows=[1])
patchseq_genes_info= pd.read_csv('../dat/1908/20190802_patchseq_genesets.csv',nrows=1)

import csv 

with open('../dat/1908/20190802_patchseq_genesets.gmt','w') as f:
    tsv_writer = csv.writer(f, delimiter='\t')
    for i in patchseq_genes.columns: #[~patchseq_genes.columns.str.startswith('T2D')]
        x=patchseq_genes[i]
        x=x[~pd.isnull(x)].tolist()
        x.insert(0,patchseq_genes_info[i][0])
        x.insert(0,i)
        tsv_writer.writerow(x)


## alpha

In [21]:
gene_exp_alpha = pd.read_csv("../dat/figdata/fig2_prom_ttest_res.csv",index_col=1)
#gene_exp_alpha.head()
df = gene_exp_alpha.loc[gene_exp_alpha["celltype"]=="alpha"].sort_values(by='odds', ascending=False)["odds"]
df=np.log2(df)

# replace inf to max exclude inf
#df=df.replace(np.inf,df[~df.isin([np.inf])].max(0))
#df=df.replace(-np.inf,df[~df.isin([-np.inf])].min(0))
df.to_csv('../dat/figdata/res.genes.a.rnk',sep='\t')
rnk = pd.read_table("../dat/figdata/res.genes.a.rnk", header=None)

for i in  rnk.index[(rnk[1]==np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100)

for i in  rnk.index[(rnk[1]==-np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100)

#rnk.set_index(0)
#rnk.head()

# run gsea
# enrichr libraries are supported by gsea module. Just provide the name

gs_res_a = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                 gene_sets='../dat/1908/20190802_patchseq_genesets.gmt', # enrichr library names or gmt file
                 #set permutation_type to phenotype if samples >=15
                 permutation_num=1000, # reduce number to speed up test
                 outdir=None,  # do not write output to disk
                 no_plot=True, # Skip plotting
                 #weighted_score_type=1,
                 #ascending=False,
                    seed=1000,
                 min_size=1,
                 max_size=2100,
                 processes=4)
                 #format='png')
gs_res_a.res2d

2019-08-06 11:12:28,593 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
multi_correlated,0.378634,1.221979,0.180108,0.388511,57,54,WDR59;F3;BMP5;ATP1B1;SLC30A8;CMC1;CCDC57;CHGA;...,WDR59;F3;BMP5;ATP1B1;SLC30A8;CMC1;CCDC57;CHGA;...
ND_exocytosis_correlated,0.260495,1.005463,0.463646,0.468528,311,282,ST8SIA3;SLC27A5;MBTPS1;ELMO2;GDA;FAM135B;CADM1...,ST8SIA3;SLC27A5;MBTPS1;ELMO2;GDA;FAM135B;CADM1...
ND_exocytosis_correlated_filtered,0.292387,1.081478,0.340807,0.495948,182,167,ST8SIA3;MBTPS1;CADM1;BMP5;BRAP;FOCAD;MAPK10;CT...,ST8SIA3;MBTPS1;CADM1;BMP5;BRAP;FOCAD;MAPK10;CT...
ND_exocytosis_anticorrelated_filtered,-0.216060,-1.047423,0.340206,0.584634,191,175,PSMB7;C1orf43;AP1B1;IP6K2;ADAM10;COG1;SLC9A3R1...,PDHA1;CLPP;ATP6V1B2;GHITM;TKT;SYVN1;TXN2;MRPL4...
mutli_anticorrelated,-0.254928,-0.979048,0.496183,0.631313,54,51,PSMB7;SLC9A3R1;RBP4;GET4;ATP5B;MAGEH1;EIF4EBP1...,SCAND1;MTHFD2;RSF1;SNRPB;TCEB2;COPS3;SNRNP40;N...
ND_exocytosis_anticorrelated,-0.208678,-1.048462,0.273973,0.872360,250,227,MYSM1;PSMB7;C1orf43;AP1B1;IP6K2;ADAM10;TPST1;C...,PDHA1;DUSP12;CLPP;ATP6V1B2;GHITM;TKT;SYVN1;TXN...
T2D_exocytosis_correlated,-0.153726,-0.631676,0.981982,0.988981,77,66,UQCC1;TTR;APBB3;SCGB2A1;DLG5;CHGA;FXYD6;CELF3;...,KMT2A;DDX46;DCTPP1;RGS9;ECH1;PIK3C2A;RTBDN;EXT...
T2D_exocytosis_anticorrelated,-0.202195,-1.059202,0.250000,1.000000,449,423,ERI3;CSNK2A2;KMT2D;SND1;AKR1B1;GRAMD1A;CHID1;T...,PUM1;TAGLN2;SH3BGRL3;TASP1;PIK3R4;NFRKB;CSTB;O...


## beta

In [22]:
gene_exp_alpha = pd.read_csv("../dat/figdata/fig2_prom_ttest_res.csv",index_col=1)
#gene_exp_alpha.head()
df = gene_exp_alpha.loc[gene_exp_alpha["celltype"]=="beta"].sort_values(by='odds', ascending=False)["odds"]
df=np.log2(df)

# replace inf to max exclude inf
#df=df.replace(np.inf,df[~df.isin([np.inf])].max(0))
df=df.replace(-np.inf,df[~df.isin([-np.inf])].min(0))
df.to_csv('../dat/figdata/res.genes.b.rnk',sep='\t')
rnk = pd.read_table("../dat/figdata/res.genes.b.rnk", header=None)

for i in  rnk.index[(rnk[1]==np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100)

for i in  rnk.index[(rnk[1]==-np.inf).tolist()].tolist():
    rnk.iloc[i,1]= df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100)

#rnk.set_index(0)
#rnk.head()


# run gsea
# enrichr libraries are supported by gsea module. Just provide the name

gs_res = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                 gene_sets='../dat/1908/20190802_patchseq_genesets.gmt', # enrichr library names or gmt file
                 #set permutation_type to phenotype if samples >=15
                 permutation_num=1000, # reduce number to speed up test
                 outdir=None,  # do not write output to disk
                 no_plot=True, # Skip plotting
                 #weighted_score_type=1,
                 #ascending=False,
                    seed=1000,
                 min_size=10,
                 max_size=2100,
                 processes=8)
                 #format='png')
gs_res.res2d

2019-08-06 11:12:36,966 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
mutli_anticorrelated,-0.405450,-1.587787,0.005952,0.029703,54,51,GET4;SLC9A3R1;RBP4;DHPS;ID2;H1F0;MAGEH1;SNF8;S...,SEC13;ATP5B;DLD;RSF1;JTB;MRFAP1L1;DNAJC8;TCEB2...
ND_exocytosis_anticorrelated,-0.286355,-1.353021,0.125000,0.074257,250,227,PDCD10;EIF3D;C14orf166;RNF14;HSPA9;DDX24;CSRP1...,SAT1;PSAP;ATP6V1B2;RSL24D1;NOL8;KPNA1;RUVBL2;Z...
ND_exocytosis_anticorrelated_filtered,-0.261807,-1.278746,0.041667,0.083168,191,175,PDCD10;EIF3D;C14orf166;RNF14;HSPA9;DDX24;CSRP1...,COPS3;MEAF6;DIABLO;SNRNP40;DPH3;MRPL9;SAT1;PSA...
ND_exocytosis_correlated_filtered,0.358500,1.195764,0.145833,0.370580,182,166,NCOR2;INS;CTBP1;KCNMB2;PALLD;CADM1;TMEM200A;MA...,NCOR2;INS;CTBP1;KCNMB2;PALLD;CADM1;TMEM200A;MA...
multi_correlated,0.399294,1.195854,0.219793,0.740494,57,54,KCNMB2;WDR59;SLC30A8;AP3B1;PIGX;DMKN;F3;BMP5;S...,KCNMB2;WDR59;SLC30A8;AP3B1;PIGX;DMKN;F3;BMP5;S...
ND_exocytosis_correlated,0.298730,1.018447,0.452165,0.756282,311,282,NCOR2;INS;CTBP1;KCNMB2;PALLD;CADM1;TMEM200A;UB...,NCOR2;INS;CTBP1;KCNMB2;PALLD;CADM1;TMEM200A;UB...
T2D_exocytosis_correlated,0.178887,0.560799,0.981693,0.993329,77,67,APBB3;SLC1A4;DLK1;CELF3;CDK5RAP1;CHGA;SCGB2A1;...,APBB3;SLC1A4;DLK1;CELF3;CDK5RAP1;CHGA;SCGB2A1
T2D_exocytosis_anticorrelated,0.167887,0.578463,0.998999,1.000000,449,423,PGM1;KMT2D;SMPD4;PRKAG2;MKLN1;KDM4C;TXNDC11;DD...,PGM1;KMT2D;SMPD4;PRKAG2;MKLN1;KDM4C;TXNDC11;DD...


### Save results

In [24]:
gs_res.res2d[['es','nes','pval','fdr','geneset_size','matched_size']].to_csv('../dat/figdata/GSEA_beta_Patchseq_seed1000.csv')
gs_res_a.res2d[['es','nes','pval','fdr','geneset_size','matched_size']].to_csv('../dat/figdata/GSEA_alpha_Patchseq_seed1000.csv')

from gseapy.plot import gseaplot, heatmap
terms = gs_res.res2d.index 
for i in range(len(terms)):
    gseaplot(gs_res.ranking, term=terms[i], **gs_res.results[terms[i]],ofname=terms[i]+'_b_seed1000.pdf')
terms = gs_res_a.res2d.index 
for i in range(len(terms)):
    gseaplot(gs_res_a.ranking, term=terms[i], **gs_res_a.results[terms[i]],ofname=terms[i]+'_a_seed1000.pdf')